In [1]:
import transformers
import pandas as pd
import numpy as np
import torch
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizerFast, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_metric
import pandas as pd

/gpfs/space/home/shuva/miniconda3/envs/roberta/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-08 14:56:19.073031: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 14:56:34.380936: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-08 14:56:34.381077: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [2]:
! pip install seqeval
! pip install datasets

In [41]:

os.environ["WANDB_API_KEY"] = api_key
os.environ["WANDB_ENTITY"] = "Hendrik"
os.environ["WANDB_PROJECT"] = "health_xlmr"
os.environ["WAND_MODE"] = "offline"

In [3]:
! ls data

README.md	fine_tuning  output_all_tags.tsv
est_output.tsv	output.tsv   training_data_AD_and_AP_1st_and_2nd.csv


In [4]:
from transformers import pipeline
#unmasker = pipeline('ner', model='xlm-roberta-base')
#unmasker("Hello I'm a <mask> model.")

In [5]:
import csv
with open('data/output_all_tags.tsv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter="\t")
    output = list(reader)

output = [row for row in output]
# labeled_tokens = [(token, label) for token, label in output]

In [6]:
total = []
temporary = []

for entry in output:
    if len(entry) == 2:
        text = entry[0]
        label = entry[1]
        if label == '0':
            label = 'O'
        if text not in ['SEP', 'CLS']:
            temporary.append([text, label])
    if len(entry) == 0:
        total.append(temporary)
        temporary = []

In [7]:
labels_ = []

for x in total:
    for y in x:
        label_ = y[1]
        labels_.append(label_)

In [8]:
nplabels = np.array(labels_)

In [9]:
unique_values = np.unique(nplabels, return_counts=True)

In [10]:
counts_ = unique_values[1]
names_ = unique_values[0]

for k in range(len(counts_)):
    print(names_[k], "-", counts_[k])

B-ADE - 946
B-Dosage - 4220
B-Drug - 16025
B-Duration - 592
B-Form - 6647
B-Frequency - 6272
B-Reason - 3776
B-Route - 5469
B-Strength - 6686
I-ADE - 774
I-Dosage - 3584
I-Drug - 4205
I-Duration - 1003
I-Form - 2907
I-Frequency - 9411
I-Reason - 3119
I-Route - 426
I-Strength - 7150
O - 544824


In [11]:
label_list = list(names_)
print(label_list)

['B-ADE', 'B-Dosage', 'B-Drug', 'B-Duration', 'B-Form', 'B-Frequency', 'B-Reason', 'B-Route', 'B-Strength', 'I-ADE', 'I-Dosage', 'I-Drug', 'I-Duration', 'I-Form', 'I-Frequency', 'I-Reason', 'I-Route', 'I-Strength', 'O']


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(names_))
list(le.classes_)
le.transform(['B-Drug'])

In [34]:
# Load the XLM-RoBERTa model and tokenizer
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

model = AutoModelForTokenClassification.from_pretrained('xlm-roberta-base', num_labels=len(label_list), id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', max_length=512)


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [35]:
print(len(total))
training_total = total[:int(len(total)*0.75)]
test_total = total[int(len(total)*0.75):]

print(len(training_total), len(test_total))

303
227 76


In [36]:
def get_all_tokens_and_ner_tags(texts):
    return pd.concat([get_tokens_and_ner_tags(texts)]).reset_index().drop('index', axis=1)
    
def get_tokens_and_ner_tags(texts):
    all_tokens = []
    all_entities = []
    for row in texts:
        tokens = [x[0] for x in row]
        entities = [x[1] for x in row]
        
        all_tokens.append(tokens)
        all_entities.append(entities)
    return pd.DataFrame({'tokens': all_tokens, 'ner_tags': all_entities})

def get_un_token_dataset(train_directory, test_directory):
    train_df = get_all_tokens_and_ner_tags(train_directory)
    test_df = get_all_tokens_and_ner_tags(test_directory)
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    return (train_dataset, test_dataset)

train_dataset, test_dataset = get_un_token_dataset(training_total, test_total)

In [37]:
train_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 227
})

In [31]:
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

In [38]:
task = "ner"
label_encoding_dict = label2id # taken from code but i switched the variable

def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

In [39]:
train_tokenized_datasets

for feature in train_tokenized_datasets.features:
    print(feature, len(train_tokenized_datasets[0][feature]))

tokens 3557
ner_tags 3557
input_ids 512
attention_mask 512
labels 512


In [63]:
# label_names
train_tokenized_datasets.features['ner_tags'].feature


Value(dtype='string', id=None)

In [ ]:
for i in range(len(train_tokenized_datasets[0]['ner_tags'])):
    print(train_tokenized_datasets[0]['ner_tags'][i], train_tokenized_datasets[0]['labels'][i])

In [43]:
# label_list = label2id = {"0": 0, "B-ADE": 1, "B-Drug": 2, "I-Drug": 3, "I-ADE": 4}
try:
    wandb.init(mode="disabled")
except:
    pass

In [45]:
batch_size = 16

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=12,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)


metric = load_metric("seqeval")
results_collection = []


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    

    results = metric.compute(predictions=true_predictions, references=true_labels)
    results_collection.append(results)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    


trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('xlmr_ner.model')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
/gpfs/space/home/shuva/miniconda3/envs/roberta/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 227
  Num Epochs = 12
  Inst

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.350879,0.489177,0.044349,0.081324,0.917721
2,No log,0.290707,0.548030,0.523940,0.535714,0.940421
3,No log,0.260630,0.666189,0.547488,0.601034,0.949417
4,No log,0.223075,0.803056,0.556907,0.657706,0.952837
5,No log,0.195461,0.667497,0.631083,0.648780,0.954810
6,No log,0.182257,0.817049,0.643250,0.719807,0.959571
7,No log,0.186796,0.653150,0.708006,0.679473,0.956651
8,No log,0.177357,0.647510,0.729592,0.686104,0.958150
9,No log,0.167734,0.693409,0.722527,0.707669,0.960833
10,No log,0.162745,0.732249,0.724490,0.728349,0.962832


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 76
  Batch size = 16
/gpfs/space/home/shuva/miniconda3/envs/roberta/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluatio

Saving model checkpoint to xlmr_ner.model
Configuration saved in xlmr_ner.model/config.json
Model weights saved in xlmr_ner.model/pytorch_model.bin
tokenizer config file saved in xlmr_ner.model/tokenizer_config.json
Special tokens file saved in xlmr_ner.model/special_tokens_map.json


In [55]:
results_collection[-1]

{'ADE': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 92},
 'Dosage': {'precision': 0.49038461538461536,
  'recall': 0.5151515151515151,
  'f1': 0.5024630541871921,
  'number': 99},
 'Drug': {'precision': 0.8481792717086835,
  'recall': 0.9055023923444976,
  'f1': 0.8759039629736766,
  'number': 1672},
 'Duration': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 12},
 'Form': {'precision': 0.5185185185185185,
  'recall': 0.22580645161290322,
  'f1': 0.3146067415730337,
  'number': 62},
 'Frequency': {'precision': 0.25,
  'recall': 0.4084507042253521,
  'f1': 0.31016042780748665,
  'number': 71},
 'Reason': {'precision': 0.2043343653250774,
  'recall': 0.24087591240875914,
  'f1': 0.22110552763819097,
  'number': 274},
 'Route': {'precision': 0.7592592592592593,
  'recall': 0.6833333333333333,
  'f1': 0.7192982456140351,
  'number': 120},
 'Strength': {'precision': 0.6505376344086021,
  'recall': 0.8287671232876712,
  'f1': 0.7289156626506025,
  'number': 146},
 'overall

In [46]:
pipe = pipeline("ner", model="xlmr_ner.model", tokenizer=tokenizer)

loading configuration file xlmr_ner.model/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlmr_ner.model",
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-ADE",
    "1": "B-Dosage",
    "2": "B-Drug",
    "3": "B-Duration",
    "4": "B-Form",
    "5": "B-Frequency",
    "6": "B-Reason",
    "7": "B-Route",
    "8": "B-Strength",
    "9": "I-ADE",
    "10": "I-Dosage",
    "11": "I-Drug",
    "12": "I-Duration",
    "13": "I-Form",
    "14": "I-Frequency",
    "15": "I-Reason",
    "16": "I-Route",
    "17": "I-Strength",
    "18": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-ADE": 0,
    "B-Dosage": 1,
    "B-Drug": 2,
    "B-Duration": 3,
    "B-Form": 4,
    "B-Frequency": 5,
    "B-Reason": 6,
 

In [47]:
tekstike = " ".join([x[0] for x in output if len(x) > 1])

In [50]:
test_text = " ".join([x[0] for x in test_total[1]])
res = pipe(test_text)
print(np.unique(np.array([x['entity'] for x in res]), return_counts=True))

(array(['B-Drug', 'B-Reason', 'I-Reason'], dtype='<U8'), array([36,  7,  3]))


In [67]:
test_text = " ".join([x[0] for x in test_total[1]])

In [36]:
res = pipe(test_text)

In [37]:
# when they were 0
for i in res:
    print(i)

{'entity': 'B-ADE', 'score': 0.99907255, 'index': 1, 'word': '▁Tu', 'start': 0, 'end': 2}
{'entity': 'B-ADE', 'score': 0.9994235, 'index': 2, 'word': 'imus', 'start': 2, 'end': 6}
{'entity': '0', 'score': 0.9999534, 'index': 3, 'word': '▁es', 'start': 7, 'end': 9}
{'entity': '0', 'score': 0.9999505, 'index': 4, 'word': 'ci', 'start': 9, 'end': 11}
{'entity': '0', 'score': 0.9999536, 'index': 5, 'word': 'talo', 'start': 11, 'end': 15}
{'entity': '0', 'score': 0.9999602, 'index': 6, 'word': 'pra', 'start': 15, 'end': 18}
{'entity': '0', 'score': 0.9999417, 'index': 7, 'word': 'mist', 'start': 18, 'end': 22}
{'entity': '0', 'score': 0.9999856, 'index': 8, 'word': '▁Pro', 'start': 23, 'end': 26}
{'entity': '0', 'score': 0.99998724, 'index': 9, 'word': 'ovi', 'start': 26, 'end': 29}
{'entity': '0', 'score': 0.9999887, 'index': 10, 'word': 'da', 'start': 29, 'end': 31}
{'entity': '0', 'score': 0.999987, 'index': 11, 'word': '▁5', 'start': 32, 'end': 33}
{'entity': '0', 'score': 0.999987, 'in

In [90]:
test_text = " ".join([x[0] for x in test_total[4]])
res = pipe(test_text)

In [53]:
def retrieve_spans_from_ner(ner_results: str):
    total = len(ner_results)
    counter = 0
    results = []

    while counter < total:
        spanstart = ner_results[counter]['start']
        spanend = ner_results[counter]['end']
        word = ner_results[counter]['word']
        entity = ner_results[counter]['entity']

        nextword = word

        for i in range(counter+1, total):
            if ner_results[i]['word'].startswith('##'):
                nextword += ner_results[i]['word']
                spanend = ner_results[i]['end']
                if ner_results[i]['entity'] != '0' and entity == '0':
                    entity = ner_results[i]['entity']
                counter += 1
            else:
                break
        results.append({'word': nextword.replace("##", ""), 'start': spanstart, 'end': spanend, 'entity': entity})
        counter += 1

    return results
